# Predicting Return on Investment (ROI) of Consumer Loans

**Andrew Nicholls** | Email: andrew.s.nicholls@gmail.com | [Github](https://github.com/Booleans)

If you are viewing this notebook on Github I recommend using the following nbviewer link instead to ensure proper formatting and working interactive charts.

[nbviewer link](https://nbviewer.jupyter.org/github/Booleans/Lending-Club-Loan-Analysis/blob/master/Loan_Analysis_Regression.ipynb)

## Summary

**Contents:**

1. [Problem Definition and Background Information](#1)
2. [Data Preparation: Wrangling, Cleaning, and Feature Extraction](#2)
3. [Exploratory Data Analysis](#3)
4. [Machine Learning Models](#4)
5. [Results and Findings](#4)
<a id='1'></a>

# 1. Introduction

### Problem Definition

LendingClub Corporation operates as an online marketplace that connects borrowers and investors in the United States. Its marketplace facilitates various types of loan products for consumers and small businesses, including unsecured personal loans, super prime consumer loans, unsecured education and patient finance loans, and unsecured small business loans. The company also offers investors an opportunity to invest in a range of loans based on term and credit characteristics. However, many loans issued through Lending Club end up being defaulted on by the borrower. The goal of this notebook is to examine the historical loan data available in order to generate a model to predict the return on investment that a new loan will generate.

### Files Provided

Lending Club provides CSV files of historical data for its loans. These files contain complete loan data for all loans issued through the time period stated, including the loan status (Current, Late, Fully Paid, etc.) and latest payment information. The files can be acquired on the [Lending Club Statistics Page](https://www.lendingclub.com/info/download-data.action). As of the creation of this project the latest data available from Lending Club was for Q1 2018.

For information on the definitions of fields contained within the historical data please see the [Lending Club Data Dictionary](https://github.com/Booleans/consumer-loan-survival-analysis/blob/master/data/LCDataDictionary.xlsx?raw=true).

The LoanStats3a file also contains information on loan applications that were declined and never issued. I have discarded those rows of data as they are not relevant to predicting loan defaults. I have also removed the last 2 rows of every spreadsheet, as they contained aggregate information on the number of loans in the file.

In [1]:
import datetime
import pickle
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from datetime import datetime as dt
from io import BytesIO
import boto3
import multiprocessing as mp

%run src/columns.py
%run src/data-cleaning.py
%run src/feature-engineering.py
%run src/payments.py
%run src/modeling.py

<a id='2'></a>
# 2. Data Preparation: Wrangling, Cleaning, and Feature Extraction

For this project I stored my data on an AWS S3 bucket called `loan-analysis-data`. I've written a function below that will allow me to easily load `csv` and `pkl.bz2` pickle files from my bucket. This enables me to perform my analysis on an AWS EC2 instance instead of working on my local machine.

In [2]:
def load_data_from_s3(filename, format='csv'):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket='loan-analysis-data', Key=filename)
    data = obj['Body'].read()
    f = BytesIO(data)
    if format=='csv':
        df = pd.read_csv(f, low_memory=False)
    if format=='pkl.bz2':
        df = pd.read_pickle(f, compression='bz2')
    return df

Reading in raw data, for now just use the pickle

In [3]:
loans = pd.read_pickle('data/raw_dataframe.pkl.bz2')

In [ ]:
loans = drop_loan_status(loans)
loans = drop_joint_applicant_loans(loans)
loans = fix_rate_cols(loans)
loans.dropna(subset=['issue_d'], inplace=True)
loans = fix_date_cols(loans)
loans = exclude_loans_before_2010(loans)
loans = clean_loan_term_col(loans)
loans = only_include_36_month_loans(loans)
loans = clean_employment_length(loans)
loans = create_missing_data_boolean_columns(loans)
loans = fill_nas(loans, value=-99)
loans = add_supplemental_rate_data(loans)
loans = create_rate_difference_cols(loans)
loans = create_months_since_earliest_cl_col(loans)
loans = change_data_types(loans)
loans.set_index('id', inplace=True)

At this point I will save the dataframe to use for exploratory data analysis. After that we will be creating dummy cols for categorical variables as well as dropping columns that are no longer necessary.

In [ ]:
loans.to_pickle('data/cleaned_loans_for_EDA.pkl.bz2', compression='bz2')

In [ ]:
loans = create_dummy_cols(loans)
loans = drop_unnecessary_cols(loans)
loans.to_pickle('data/cleaned_loans.pkl.bz2', compression='bz2')

#### Return on Investment Calculation for Completed Loans

The goal of our model will be to predict return on investment of a loan. To do this, we need to get the payments data for all loans in our training dataset. Once we've acquired all payment information, we can calculate return on investment.

In [3]:
cleaned_loans = pd.read_pickle('data/cleaned_loans.pkl.bz2', compression='bz2')

The raw payments data is contained in the file `PMTHIST_INVESTOR_201808.csv` stored in my S3 bucket. Raw payments data can be found on the [additional statistics](https://www.lendingclub.com/company/additional-statistics) page on Lending Club's website.

Once the raw data is loaded, we need to clean it using the function `get_cleaned_payment_history_data` from the `payments.py` file. 

In [4]:
payments = load_data_from_s3('PMTHIST_INVESTOR_201808.csv', format='csv')

In [5]:
cleaned_payments = get_cleaned_payment_history_data(payments)

/home/ubuntu/consumer-loan-survival-analysis/src/payments.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['RECEIVED_D'] = convert_payment_date(df['RECEIVED_D'])
/home/ubuntu/consumer-loan-survival-analysis/src/payments.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['IssuedDate'] = convert_payment_date(df['IssuedDate'])
/home/ubuntu/consumer-loan-survival-analysis/src/payments.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Now that we've got the cleaned payments data, let's save it as a pickle file for easy access in the future. 

In [10]:
cleaned_payments.to_pickle('data/cleaned_payments_data.pkl.bz2', compression='bz2')

#### Training Data

Before we calculate ROI for the training data, we need to define what the training data is. We can only calculate the ROI for loans that have completed. Since these are 36 month loans, our training data will be all loans issued before September 2015. Loans issued before that date are guaranteed to have been completed by September 2018, the current date.

In [7]:
training_loans, testing_loans = get_training_and_testing_data(cleaned_loans)
training_loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,issue_d,dti,delinq_2yrs,fico_range_low,fico_range_high,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
id,,,,,,,,,,,,,,,,,,,,,
1077501,5000.0,10.650000,162.869995,10.0,24000.0,2011-12-01,27.65,0,735.0,739.0,...,0,0,0,0,0,0,0,0,0,0
1077175,2400.0,15.960000,84.330002,10.0,12252.0,2011-12-01,8.72,0,735.0,739.0,...,0,0,0,0,0,0,0,1,0,0
1076863,10000.0,13.490000,339.309998,10.0,49200.0,2011-12-01,20.00,0,690.0,694.0,...,0,0,0,0,0,1,0,0,0,0
1075269,5000.0,7.900000,156.460007,3.0,36000.0,2011-12-01,11.20,0,730.0,734.0,...,0,0,0,0,0,0,0,0,0,1
1072053,3000.0,18.639999,109.430000,9.0,48000.0,2011-12-01,5.35,0,660.0,664.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
training_loan_ids = training_loans.index[:10]
training_loan_ids

Index([1077501, 1077175, 1076863, 1075269, 1072053, 1069908, 1064687, 1069866,
       1069057, 1069759],
      dtype='object', name='id')

In [11]:
training_payments = get_relevant_payments(cleaned_payments, train_ids)

In [8]:
def get_relevant_payments(all_payments, loan_ids_from_training_set):
    cols = ['RECEIVED_AMT_INVESTORS', 'mths_since_issue']
    return all_payments.loc[pd.IndexSlice[:, loan_ids_from_training_set], :][cols]

In [14]:
training_payments = get_relevant_payments(cleaned_payments, train_ids)

In [12]:
training_payments

RECEIVED_AMT_INVESTORS  mths_since_issue
RECEIVED_D LOAN_ID                                          
2012-02-01 1064687              305.380005                 2
           1069057              325.739990                 2
           1069759               35.310001                 2
           1069866               96.680000                 2
           1069908              402.540009                 2
           1072053              109.430000                 2
           1075269              156.460007                 2
           1076863              339.309998                 2
           1077175               84.330002                 2
           1077501              162.055649                 2
2012-03-01 1064687              305.380005                 3
           1069057              325.739990                 3
           1069759               35.310001                 3
           1069866               96.680000                 3
           1069908              402.540009                 3
           1072053              109.430000                 3
           1075269              156.460007                 3
           1076863              339.309998                 3
           1077175               84.330002                 3
           1077501              162.055649                 3
2012-04-01 1064687              305.380005                 4
           1069057              325.739990                 4
           1069759               35.310001                 4
           1069866               96.680000                 4
           1069908              402.540009                 4
           1072053              109.430000                 4
           1075269              156.460007                 4
           1076863              339.309998                 4
           1077175               84.330002                 4
           1077501              162.055649                 4
...                                    ...               ...
2014-09-01 1069759               35.310001                33
           1069866               96.680000                33
           1072053              109.430000                33
           1075269              156.460007                33
           1076863              339.309998                33
           1077501              162.055649                33
2014-10-01 1069759               35.310001                34
           1069866               96.680000                34
           1072053              109.430000                34
           1076863              339.309998                34
           1077501              162.055649                34
2014-11-01 1069759               35.310001                35
           1069866               96.680000                35
           1072053              109.430000                35
           1075269              156.460007                35
           1075269              156.460007                35
           1076863              339.309998                35
           1077501              162.055649                35
2014-12-01 1069759               35.310001                36
           1069866               96.680000                36
           1072053              109.430000                36
           1075269              156.460007                36
           1076863              339.309998                36
           1077501              162.055649                36
2015-01-01 1069759               36.320000                37
           1069866              102.430000                37
           1072053              111.339996                37
           1075269              161.029999                37
           1076863              357.480011                37
           1077501              170.761902                37

[292 rows x 2 columns]